In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [2]:
sc = SparkContext(master='local', appName='TransformacionesYAcciones')

In [3]:
rdd1 = sc.parallelize([1,2,3])
type(rdd1)

pyspark.rdd.RDD

In [4]:
rdd1.collect()

[1, 2, 3]

In [5]:
sc

<SparkContext master=local appName=TransformacionesYAcciones>

In [6]:
!ls files

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [7]:
#aca cargo la ruta del archivo
path = "files/"

In [8]:
equiposOlimpicosRDD = sc.textFile(path+'paises.csv') \
    .map(lambda line : line.split(','))

In [9]:
equiposOlimpicosRDD.take(5)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG']]

In [10]:
#sc.stop() #stop detiene el contexto

In [11]:
equiposOlimpicosRDD.map(lambda x: (x[2])).distinct().count()

231

In [12]:
equiposOlimpicosRDD.map(lambda x: (x[2], x[1])).groupByKey()\
    .mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [13]:
equiposOlimpicosRDD.map(lambda x: (x[2], x[1])).groupByKey()\
    .mapValues(list).take(5)

[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise']),
 ('MEX',
  ['A North American Team',
   'Acipactli',
   'Chamukina',
   'Mexico',
   'Mexico-1',
   'Mexico-2',
   'Nausikaa 4',
   'Tlaloc',
   'Xolotl']),
 ('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan'])]

In [14]:
equiposArgentinos = equiposOlimpicosRDD.filter(lambda l: 'ARG' in l)

In [15]:
equiposArgentinos.collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

In [16]:
equiposOlimpicosRDD.count()

1185

In [17]:
equiposOlimpicosRDD.countApprox(20)

1185

In [18]:
!ls files

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [19]:
deportistaOlimpicoRDD = sc.textFile(path+'deportista.csv').map(lambda l: l.split(','))
deportistaOlimpicoRDD2 = sc.textFile(path+'deportista2.csv').map(lambda l: l.split(','))

In [20]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpicoRDD2)

In [21]:
deportistaOlimpicoRDD.count()

135572

In [22]:
equiposOlimpicosRDD.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [23]:
deportistaOlimpicoRDD.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [24]:
deportistaOlimpicoRDD.map(lambda l: [l[-1], l[:-1]]).join(equiposOlimpicosRDD.map(lambda x: [x[0], x[2]])).take(5)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN')),
 ('199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'CHN')),
 ('199', (['1463', 'Ai Linuer', '1', '25', '160', '62'], 'CHN')),
 ('199', (['1464', 'Ai Yanhan', '2', '14', '168', '54'], 'CHN'))]

In [25]:
deportistaOlimpicoRDD.map(lambda l: [l[-1], l[:-1]]).join(equiposOlimpicosRDD.map(lambda x: [x[0], x[2]])).takeSample(False, 10, 25)

[('273',
  (['56134', 'Niels Christian Kold Jrgensen', '1', '21', '0', '0'], 'DEN')),
 ('624', (['13109', 'Harry Arthur Bonavia', '1', '20', '0', '0'], 'MLT')),
 ('399', (['98241', 'Eberhard Radzik', '1', '25', '168', '75'], 'GER')),
 ('96', (['83781', 'Albert Muylle', '1', '0', '0', '0'], 'BEL')),
 ('1096',
  (['101476', 'Clarence Franklin Robison', '1', '25', '192', '77'], 'USA')),
 ('967', (['18615', 'Lesley Carstens', '2', '27', '0', '0'], 'RSA')),
 ('362', (['78848', 'mile Mercier', '1', '0', '0', '0'], 'FRA')),
 ('810',
  (['94563', 'Andrzej Ryszard Pitkowski', '1', '22', '169', '68'], 'POL')),
 ('1096', (['114820', 'Jerome Steinert', '1', '28', '0', '0'], 'USA')),
 ('576', (['32099', 'Assaf ElMurr', '1', '24', '0', '0'], 'LIB'))]

In [26]:
resultados = sc.textFile(path+'resultados.csv').map(lambda x: x.split(','))

In [27]:
resultadoGanador = resultados.filter(lambda l : 'NA' not in l[1])

In [28]:
resultadoGanador.take(2)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4']]

In [29]:
deportistaPaises  = deportistaOlimpicoRDD.map(lambda l: [l[-1], l[:-1]]) \
    .join(equiposOlimpicosRDD.map(lambda x: [x[0], x[2]])) 
        

In [30]:
deportistaPaises.join(resultadoGanador).take(5)

[('74',
  ((['65', 'Patimat Abakarova', '2', '21', '165', '49'], 'AZE'), 'Gold')),
 ('74', ((['129', 'Ruslan Abbasov', '1', '22', '181', '74'], 'AZE'), 'Gold')),
 ('74', ((['130', 'Tural Abbasov', '1', '18', '182', '76'], 'AZE'), 'Gold')),
 ('74', ((['131', 'Tran Abbasova', '2', '33', '159', '53'], 'AZE'), 'Gold')),
 ('74',
  ((['335', 'Abdulqdir Abdullayev', '1', '28', '188', '91'], 'AZE'), 'Gold'))]

In [31]:
valoresMedallas = {
    'Gold' : 7,
    'Silver' : 5,
    'Bronze' : 4
}

In [50]:
paisesMedallas = deportistaPaises.join(resultadoGanador)

In [51]:
paisesMedallas.map(lambda x: x[1][0][-1], x[1][1]]).take(4)

[['AZE', 'Gold'], ['AZE', 'Gold'], ['AZE', 'Gold'], ['AZE', 'Gold']]

In [56]:
paisesMedallas = paisesMedallas.map(lambda x: (x[1][0][-1], valoresMedallas[x[1][1]]))

In [57]:
from operator import add

In [58]:
conclusion = paisesMedallas.reduceByKey((add)).sortBy(lambda x: x[1], ascending=False)

In [60]:
conclusion.take(10)

[('CAN', 32538),
 ('ARG', 12520),
 ('HUN', 10860),
 ('MEX', 6124),
 ('RSA', 3788),
 ('BLR', 3580),
 ('LTU', 1535),
 ('MGL', 1460),
 ('USA', 1342),
 ('AZE', 1218)]